In [1]:
!pip install -q rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 23.4 MB/s eta 0:00:00


# Build Molecule from Matrix

- Our GAN will be outputting adjacency matrices referencing molecules. We can use the matrix to create a molecule from the matrix.
    - Iterate through the matrix adding our atoms and bonds

In [ ]:
from rdkit import Chem

def adjacency_matrix_to_mol(matrix, atom_type = 6):
  mol = Chem.RWMol()
  atom_map = {}

  # Step 1: Add Atoms
  for i in range(len(matrix)):
    atom = Chem.Atom(atom_type)
    mol_idx = mol.AddAtom(atom)
    atom_map[i] = mol_idx

  # Step 2: Add Bonds
  for i, row in enumerate(matrix):
    for j, value in enumerate(row):
      if i < j and value != 0:
        bond_type = Chem.BondType.SINGLE

  return mol

- Code Explanation
    - We default the atom_type value to 6, the atomic number for carbon
    - Chem.RWMol is an editable molecule
    - atom_map variable will map from matrix index to atom index
    - Adding Atoms
        - we create an atom of the correct atom type
        - we index the atom added to the molecule and add it to the map
    - Adding Bonds
        - Iterate through the matrix and add necessary bonds
- This is a very basic way to convert from matrix to molecule. Additional functionality will be added down the road to handle multiple bond types and multiple atom types.

# Validity

- “Validity is the fraction of correctness that a SMILES string translates to a real structure. Low validity is indicative of a poorly behaving model that has struggled to learn the SMILES grammar.” - https://pmc.ncbi.nlm.nih.gov/articles/PMC10664602/#Sec2
- Since the model is being created from the matrix using RWMol (Read-Write Molecule), we have to manually check for validity.

In [ ]:
def validity(mols):
  val_mols = {}

  for i, mol in enumerate(mols):
    if mol is None:
      continue

    valid = True

    # Sanitize
    try:
      Chem.SanitizeMol(mol)
    except Exception:
      continue

    # Check Kekulization
    try:
      Chem.Kekulize(mol, clearAromaticFlags = True)
    except Exception:
      continue

    # Check Valency
    for atom in mol.GetAtoms():
      explicit_valence = atom.GetExplicitValence()
      if explicit_valence > atom.GetTotalValence():
        valid = False
        break

    if valid:
      val_mols[i] = mol

  valid_score = len(val_mols) / len(mols) if len(mols) > 0 else 0
  return valid_score, val_mols

- Code Explanation
    - To check manually for validity, we have to check sanitization, kekulization, and valency. To do so, we use a set of try, except clauses in the code for the sanitization and kekulization portions. If SanitizeMol or Kekulize doesn’t work, we continue to next iteration and know the molecule is invalid.
    - For valency, we have to compare explicit and total valency for each atom in the molecule. If any fail the test, the molecule is invalid.
    - If the molecule is valid, we add it to the list. Once we are done iterating through our molecules, we find the validity score, ensuring to avoid division by 0.

# Uniqueness

- “Uniqueness is the fraction of unique molecules, where non-unique molecules are defined as having canonical SMILES that match those previously sampled or in the same batch. Low uniqueness is indicative of a poorly behaving model that is ‘stuck’ in a particular region of chemical space.” - https://pmc.ncbi.nlm.nih.gov/articles/PMC10664602/#Sec2
- percentage of unique compounds among valid compounds

In [ ]:
from rdkit.Chem import rdFingerprintGenerator

def morg_fp(mol):
  fp_gen = rdFingerprintGenerator.GetMorganGenerator(
                                                    radius = 2,
                                                    fpSize = 2048
                                                    )
  return fp_gen.GetFingerprint(mol)

def uniqueness(val_mol_list):
  fps = [morg_fp(mol) for mol in val_mol_list]
  unique_fps = set(fps)
  u = len(unique_fps) / len(val_mol_list) if len(val_mol_list) > 0 else 0
  return u , unique_fps


- Code Explanation:
    - When checking for uniqueness, checking the Morgan Fingerprint is the most common way of doing so. Thus, we define a function to get the Morgan Fingerprint for a molecule. Then, we run each molecule for its fingerprint and use set() to extract all unique molecules from our list of valid molecules.
    - Once we have that, we can easily calculate the uniqueness score.

# Novelty

- “Novelty is the ratio of valid, unique canonical SMILES not present in the training dataset, and low novelty indicates the model cannot generalize beyond the training data” - https://pmc.ncbi.nlm.nih.gov/articles/PMC10664602/#Sec2

In [ ]:
def novelty(train_mols_list, unique_mols_list):
  unknown = {}
  train_smiles = {Chem.MolToSmiles(mol, isomericSmiles = True) for mol in train_mols_list if mol}

  for i, mol in enumerate(unique_mols_list):
    if mol is None:
      continue

    mol_smiles = Chem.MolToSmiles(mol, isomericSmiles = True)

    if mol_smiles in train_smiles:
      continue
    else:
      unknown[i] = mol

    if unique_mols_list:
      n_score = len(unknown) / len(unique_mols_list)
      return n_score, unknown
    else:
      return None, None

- Code Explanation
    - For novelty, we must first convert the train molecules to smiles for direct comparison. We then enter the for loop where we convert each unique molecule into smiles iteratively. If it is not in the training, it is unknown and added to the list. We then calculate the novelty score and return.